In [1]:
import urllib
import re
from bs4 import BeautifulSoup
import time
import os.path
from os import path
import lxml

In [2]:
#genre_list=["sculpture","portrait","nude painting (nu)"]
genre_list=[ "sculpture"]

In [3]:
for genre in genre_list:
    file_path = 'D:/kozmiksefer/disruption/data_set/'+str(genre)
    print(file_path)
    if path.exists(file_path) == False:
      os.makedirs(file_path)
      os.chdir(file_path)



D:/kozmiksefer/disruption/data_set/sculpture


In [4]:
base_url = "https://www.wikiart.org"
# iterate through all artists by last name alphabetically
# like this https://www.wikiart.org/en/Alphabet/m/text-list
for c in range(ord('j'), ord('z') + 1):
  char = chr(c)
  artist_list_url = base_url + "/en/Alphabet/" + char + "/text-list"
  print(artist_list_url)

  genre_soup = BeautifulSoup(urllib.request.urlopen(artist_list_url), "lxml")
  artist_list_main = genre_soup.find("main")
  lis = artist_list_main.find_all("li")

  # for each list element
  for li in lis: 
    born = 0
    died = 0

    # get the date range
    for line in li.text.splitlines():
      if line.startswith(",") and "-" in line:
        parts = line.split('-')
        if len(parts) == 2:
          born = int(re.sub("[^0-9]", "",parts[0]))
          died = int(re.sub("[^0-9]", "",parts[1]))

    link = li.find("a")
    artist = link.attrs["href"]

    # get the artist's main page
    artist_url = base_url + artist
    artist_soup = BeautifulSoup(urllib.request.urlopen(artist_url), "lxml")

    # only look for artists with the word abstract on their main page
    for genre in genre_list:
      if genre in artist_soup.text: 
        print(artist + " " + str(born) + " - " + str(died))
        #artist_soup.text

        # get the artist's web page for the artwork
        url = base_url + artist + '/all-works/text-list'
        artist_work_soup = BeautifulSoup(urllib.request.urlopen(url), "lxml")

        # get the main section
        artist_main = artist_work_soup.find("main")
        image_count = 0
        artist_name = artist.split("/")[2]
        # get the list of artwork
        lis = artist_main.find_all("li")

        # for each list element
        for li in lis:
          link = li.find("a")

          if link != None:
            painting = link.attrs["href"]

            # get the painting
            url = base_url + painting
            print(url)

            try:
              painting_soup = BeautifulSoup(urllib.request.urlopen(url), "lxml")

            except:
              print("error retreiving page")
              continue

            #check the genre
            genre_field = painting_soup.find("span", {"itemprop":"genre"})
            print("genre:",genre_field, "bizim aradığımıız:",genre)
            if genre_field != None and genre in genre_field:
              print("field :", genre_field)

              # get the url
              og_image = painting_soup.find("meta", {"property":"og:image"})
              image_url = og_image["content"].split("!")[0] # ignore the !Large.jpg at the end
              print(image_url)

              parts = url.split("/")
              painting_name = parts[-1]
              file_path='D:/kozmiksefer/disruption/data_set/'+str(genre)
              save_path = file_path + "/" + artist_name + "_" + painting_name + ".jpg"

              #download the file
              try:
                print("downloading to " + save_path)
                time.sleep(0.2)  # try not to get a 403                    
                urllib.request.urlretrieve(image_url, save_path)
                image_count = image_count + 1
              except Exception as e:
                print("failed downloading " + image_url, e) 
                      

https://www.wikiart.org/en/Alphabet/j/text-list
/en/alfredo-jaar 0 - 0
https://www.wikiart.org/en/alfredo-jaar/gold-in-the-morning-a-j-1985
genre: <span itemprop="genre">photo</span> bizim aradığımıız: sculpture
https://www.wikiart.org/en/alfredo-jaar/a-logo-for-america-1987
genre: <span itemprop="genre">video</span> bizim aradığımıız: sculpture
https://www.wikiart.org/en/alfredo-jaar/1-1-1-1987
genre: <span itemprop="genre">installation</span> bizim aradığımıız: sculpture
https://www.wikiart.org/en/alfredo-jaar/the-rwanda-project-the-eyes-of-gutete-emerita-1996
genre: <span itemprop="genre">installation</span> bizim aradığımıız: sculpture
https://www.wikiart.org/en/alfredo-jaar/searching-for-africa-in-life-1996
genre: <span itemprop="genre">installation</span> bizim aradığımıız: sculpture
https://www.wikiart.org/en/alfredo-jaar/lament-of-the-images-2002
genre: <span itemprop="genre">installation</span> bizim aradığımıız: sculpture
https://www.wikiart.org/en/alfredo-jaar/park-of-the-la